In [1]:
### Funcion para crear conexion

import sqlite3
from sqlite3 import Error

def create_connection(path):
  connection=None
  try:
    connection=sqlite3.connect(path)
    print("Conexion fue exitosa")
  except Error as e:
    print(f"El error fue: {e}")
  return connection

In [2]:
connection = create_connection("rodri.sqlite3")

### INSERT INTO TABLE ... VALUES (...,...,...,...)

Conexion fue exitosa


In [3]:
# Instruccion para ejecutar queries -para DDL-

def execute_query(connection,query):
  cursor = connection.cursor()
  try:
    cursor.execute(query)
    connection.commit()
    print("Query ejecutado exitosamente")
  except Error as e:
    print(f"El error fue: {e}")

In [7]:
crear_tabla_coches = """
CREATE TABLE IF NOT EXISTS Coches (
  ID integer PRIMARY KEY AUTOINCREMENT,
  Version TEXT NOT NULL,
  Marca TEXT NOT NULL,
  Color TEXT NOT NULL,
  Tipo TEXT NOT NULL
);
"""

crear_tabla_personas="""
CREATE TABLE IF NOT EXISTS Personas (
  ID integer PRIMARY KEY AUTOINCREMENT,
  Nombre TEXT NOT NULL,
  Edad integer NOT NULL
);
"""

crear_tabla_ventas = """
CREATE TABLE IF NOT EXISTS Ventas (
  ID integer PRIMARY KEY AUTOINCREMENT,
  ID_Persona INTEGER NOT NULL,
  ID_Coche INTEGER NOT NULL,
  Venta INTEGER,
  FOREIGN KEY (ID_Persona) REFERENCES Personas (ID) FOREIGN KEY (ID_Coche) REFERENCES Coches (ID)
)
"""

In [8]:
### Ejecuciones de los queries

execute_query(connection,crear_tabla_coches)
execute_query(connection,crear_tabla_personas)
execute_query(connection,crear_tabla_ventas)

Query ejecutado exitosamente
Query ejecutado exitosamente
Query ejecutado exitosamente


In [14]:
## Poblar las tablas

poblar_coches = """
INSERT INTO Coches (Version, Tipo, Marca, Color)
VALUES 
('A','NISSAN', 'Sedan', 'Rojo'),
('B','CHEV', 'Sedan', 'Azul'),
('C','AUDI', 'Hatch', 'Blanco'),
('D','BMW', 'Hatch', 'Amarillo'),
('E','AUDI', 'Coupe', 'Negro');
"""

poblar_personas = """
INSERT INTO Personas (Nombre, Edad)
VALUES
("Rodrigo",26),
("Alejandro",30),
("Nancy",28);
"""
poblar_ventas="""
INSERT INTO Ventas (ID_Persona, ID_Coche, Venta)
VALUES
(1,1,1),
(1,2,1),
(1,3,1),
(3,2,1),
(5,3,1);
"""

pd.read_csv

In [15]:
### Ejecuciones de los queries

execute_query(connection,poblar_coches)
execute_query(connection,poblar_personas)
execute_query(connection,poblar_ventas)

Query ejecutado exitosamente
Query ejecutado exitosamente
Query ejecutado exitosamente


In [16]:
## Funcion para hacer DML (SELECT) - Funcion para hacer lecturas

def execute_read_query(connection, query):
  cursor = connection.cursor()
  result = None
  try:
    cursor.execute(query)
    result = cursor.fetchall()
    print("Resultado del query exitoso")
    return result
  except Error as e:
    print(f'El error fue {e}')

In [17]:
### Ejecutar un query que nos devuelva los datos que existan referenciados entre 
### las tres tablas, es decir, todo aquello que vive a la vez en las tres.

doble_join = """
SELECT v.ID, v.ID_Persona, v.ID_Coche, c.Version, c.Marca, c.Color, c.Tipo, p.Nombre, p.Edad, v.Venta
FROM ventas as v
      INNER JOIN coches as c ON v.ID_Coche = c.ID
      INNER JOIN personas as p ON v.ID_Persona = p.ID
"""

todo_entre_tablas=execute_read_query(connection,doble_join)
todo_entre_tablas ## Una desnormalizacion de las tres tablas para formar una nueva tabla

Resultado del query exitoso


[(1, 1, 1, 'A', 'Sedan', 'Rojo', 'NISSAN', 'Rodrigo', 26, 1),
 (2, 1, 2, 'B', 'Sedan', 'Azul', 'CHEV', 'Rodrigo', 26, 1),
 (3, 1, 3, 'C', 'Hatch', 'Blanco', 'AUDI', 'Rodrigo', 26, 1),
 (4, 3, 2, 'B', 'Sedan', 'Azul', 'CHEV', 'Nancy', 28, 1)]

In [18]:
import pandas as pd
pd.DataFrame(todo_entre_tablas,columns=['v.ID', 'v.ID_Persona', 'v.ID_Coche', 'c.Version', 'c.Marca',
                                        'c.Color', 'c.Tipo', 'p.Nombre', 'p.Edad', 'v.Venta'])

,v.ID,v.ID_Persona,v.ID_Coche,c.Version,c.Marca,c.Color,c.Tipo,p.Nombre,p.Edad,v.Venta
0,1,1,1,A,Sedan,Rojo,NISSAN,Rodrigo,26,1
1,2,1,2,B,Sedan,Azul,CHEV,Rodrigo,26,1
2,3,1,3,C,Hatch,Blanco,AUDI,Rodrigo,26,1
3,4,3,2,B,Sedan,Azul,CHEV,Nancy,28,1
